In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import datetime
import tensorflow as tf

In [34]:
root_path = "/home/sridharpatil047/transfer_learning/"
img_dir = root_path+"data_final"
df = pd.read_csv(root_path+'labels_final.csv')

#df = df.head(350) ## Comment this later
df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [35]:
## converting labels to string because flow_from_dataframe requires labels to be str
df['label'] = df.label.astype(str)

df.shape

(48000, 2)

In [36]:
BATCH_SIZE = 50
EPOCHS =15
IMG_SIZE = (224, 224)

In [37]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.33,
                                                         preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_generator = datagen.flow_from_dataframe(df,directory=img_dir, x_col='path', y_col='label',subset='training', 
                                              class_mode='categorical', target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE)

valid_generator = datagen.flow_from_dataframe(df,directory=img_dir, x_col='path', y_col='label',subset='validation',
                                              class_mode='categorical', target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE)

Found 32160 validated image filenames belonging to 16 classes.
Found 15840 validated image filenames belonging to 16 classes.


In [38]:
train_generator.n, valid_generator.n

(32160, 15840)

In [39]:
train_generator.image_shape, valid_generator.image_shape

((224, 224, 3), (224, 224, 3))

# Model1

In [40]:
## code inspired from here: https://www.tensorflow.org/tutorials/images/transfer_learning
model1_path = root_path+'models/model1/'

In [41]:
## source: https://www.tensorflow.org/tutorials/images/transfer_learning#freeze_the_convolutional_base
IMG_SHAPE = IMG_SIZE + (3,)
model_VGG16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=IMG_SHAPE)
model_VGG16.trainable = False

In [42]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(model_VGG16)
model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', 
                                 kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))

model.add(tf.keras.layers.Dense(16, activation='softmax'))

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])

log_dir = model1_path + datetime.datetime.now().strftime("%m%d%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_generator,
          epochs=EPOCHS,
          validation_data=valid_generator,
         callbacks=[tensorboard_callback])
model.save(model1_path+'model1.h5')

Epoch 1/15
644/644 [==============================] - 5321s 8s/step - loss: 3.6017 - acc: 0.1340 - val_loss: 2.4150 - val_acc: 0.2413
Epoch 2/15
644/644 [==============================] - 5318s 8s/step - loss: 2.3280 - acc: 0.2929 - val_loss: 2.0986 - val_acc: 0.3812
Epoch 3/15
644/644 [==============================] - 5321s 8s/step - loss: 2.0879 - acc: 0.3704 - val_loss: 1.9347 - val_acc: 0.4155
Epoch 4/15
644/644 [==============================] - 5316s 8s/step - loss: 1.9201 - acc: 0.4152 - val_loss: 1.7983 - val_acc: 0.4568
Epoch 5/15
644/644 [==============================] - 5319s 8s/step - loss: 1.7730 - acc: 0.4606 - val_loss: 1.6172 - val_acc: 0.5028
Epoch 6/15
644/644 [==============================] - 5316s 8s/step - loss: 1.6084 - acc: 0.5042 - val_loss: 1.4807 - val_acc: 0.5481
Epoch 7/15
644/644 [==============================] - 5306s 8s/step - loss: 1.4862 - acc: 0.5389 - val_loss: 1.3910 - val_acc: 0.5805
Epoch 8/15
644/644 [==============================] - 5304s 8s